In [ ]:
%pip install ultralytics opencv-python

In [5]:
import cv2
from ultralytics import YOLO
import asyncio

In [3]:

# Initialize YOLOv8 model
model = YOLO('yolov8n.pt')  # Load the default YOLOv8 nano model

100%|██████████| 6.25M/6.25M [00:00<00:00, 96.3MB/s]


In [4]:
async def process_image(model, image):
    loop = asyncio.get_event_loop()
    return await loop.run_in_executor(None, model, image)

# Initialize the camera
cap = cv2.VideoCapture(0)  # 0 for default camera

model_busy = False

async def main():
    global model_busy
    while True:
        # Read a frame from the camera
        ret, frame = cap.read()
        if not ret:
            break

        # Run YOLOv8 inference on the frame if model is not busy
        if not model_busy:
            model_busy = True
            results = await process_image(model, frame)
            
            # Visualize the results on the frame
            annotated_frame = results[0].plot()
            
            # Display the annotated frame
            cv2.imshow("YOLOv8 Inference", annotated_frame)
            model_busy = False
        else:
            # If model is busy, display the original frame
            cv2.imshow("YOLOv8 Inference", frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    # Release the camera and close windows
    cap.release()
    cv2.destroyAllWindows()

# Run the main async function
asyncio.run(main())